# MAV - Practice 4 -> PAIS


#### Вариант 5: (a xor b) and (b xor c)

Необходимо реализовать нейронную сеть вычисляющую результат заданной логической операции. 
Затем реализовать функции, которые будут симулировать работу построенной модели. 
Функции должны принимать тензор входных данных и список весов. Должно быть реализовано 2 функции

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

Заранее создаем датасет представляющий собой таблицу истиности для логической функции из вариант.
Как раз это и описывает, что будет предсказываться NN


In [2]:
data = pd.read_csv('data/data_bin.csv')
X = data.drop(columns=['target'])
y = data['target']
data

,a,b,c,target
0,0,0,0,0
1,0,0,1,0
2,0,1,0,1
3,0,1,1,0
4,1,0,0,0
5,1,0,1,1
6,1,1,0,0
7,1,1,1,0


На самом деле модель плохо обучается на таких маленьких данных - как капля в море. Однако функции работают корректно, при простой симуляции 3 входных параметра -> 1 нейрон. Для наглядного представления. 

При синтетическом датасете в 1000+ значений, и средней модели -> результат есть

In [3]:
 # Функция создания модели нейронной сети
def create_model() -> Sequential:
    """
    Creates a neural network model for binary classification.

    The model consists of 1 input layer with shape (3,) and 1 output layer with sigmoid activation function.
    It is compiled with binary cross-entropy loss and Adam optimizer.

    Args:
        None

    Returns:
        Sequential: A compiled Keras neural network model for binary classification.

    Examples:
        >>> from tensorflow.keras.models import Sequential
        >>> baseline_model = create_model()
        >>> print(baseline_model.summary())
    """
    baseline_model: Sequential = Sequential()
    
    # Input layer with shape (3,)
    baseline_model.add(Input(shape=(3,), name='input'))
    
    # Output layer with sigmoid activation function
    baseline_model.add(Dense(1, activation='sigmoid', name='output'))

    # Compile the model with binary cross-entropy loss and Adam optimizer
    baseline_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Print the model summary
    baseline_model.summary()
    return baseline_model

In [4]:
 # Функция создает отчетность после обучения нейронной сети.
def create_error_report(model_report: Sequential, val_split: float=0.2) -> tuple[Sequential, np.ndarray]:
    """
    Creates an error report for the given model by training it on the dataset and splitting it into training and validation sets.

    Args:
        model_report (Sequential): The Keras neural network model to be trained.
        val_split (float, optional): The proportion of data to be used for validation. Defaults to 0.2.

    Returns:
        tuple[Sequential, np.ndarray]: A tuple containing the trained model and the validation loss array.

    Examples:
        >>> from tensorflow.keras.models import Sequential
        >>> from tensorflow.keras.datasets import load_breast_cancer
        >>> (X_train, y_train), (X_test, y_test) = load_breast_cancer(return_numpy=False)
        >>> model_report = create_model()
        >>> error_report, val_loss = create_error_report(model_report, val_split=0.2)

    Notes:
        The data is assumed to be stored in a CSV file named 'data/data_bin.csv' with column names matching the feature names.
        The 'target' column should contain the target variable values.
    """
    # Load the dataset from the CSV file
    data: pd.DataFrame = pd.read_csv('data/data_bin.csv')

    # Split the data into features (X) and target variable (y)
    X = data.drop(columns=['target'])
    y = data['target']

    # Train the model on the training set with validation split
    model_report.fit(X, y, epochs=100, batch_size=32, validation_split=val_split)

    return model_report

1. Функция, в которой все операции реализованы как поэлементные операции над тензорами

In [5]:
def simulate_neural_network(data_tensor: np.ndarray, weight_bias_tensor) -> tuple[np.ndarray, list]:
    """
    Simulates the neural network by performing forward pass and applying sigmoid activation function.

    Args:
        data_tensor (np.ndarray): The input tensor to be passed through the neural network.
        weight_bias_tensor (tuple[np.ndarray, np.ndarray]): A tuple containing the weights and bias tensors of the neural network.

    Returns:
        tuple[np.ndarray, list]: A tuple containing the output tensor and a list of sigmoid activation values.

    Examples:
        >>> import numpy as np
        >>> data_tensor = np.random.rand(10, 5)
        >>> weight_bias_tensor = (np.random.rand(5), np.array([1.0]))
        >>> output_tensor, sigmoid_tensor = simulate_neural_network(data_tensor, weight_bias_tensor)

    Notes:
        The neural network is assumed to have one layer with the given weights and bias.
    """
    # Get the size of the input tensor
    size: int = data_tensor.shape[0]

    # Get the number of parameters in the output tensor
    size_params: int = data_tensor.shape[1]

    # Extract the weights and bias tensors from the input tuple
    weights: np.ndarray = weight_bias_tensor[0]
    bias: np.ndarray = weight_bias_tensor[1]

    # Initialize an empty list to store sigmoid activation values
    sigmoid_tensor: list = []

    # Initialize an output tensor with zeros
    output_tensor: np.ndarray = np.zeros((size, size_params))

    # Perform forward pass for each input data point
    for i in range(size):
        for j in range(size_params):
            output_tensor[i, j] += (data_tensor[i, j] * weights[j])
        # Calculate the weighted sum of inputs and add bias
        r: np.ndarray = sum(output_tensor[i]) + bias[0]
        # Append the sigmoid activation value to the list
        sigmoid_tensor.append(np.round(1 / (1 + np.exp(-r)), 3))

    return output_tensor, sigmoid_tensor


2. Функция, в которой все операции реализованы с использованием операций над тензорами из NumPy

In [6]:
def simulate_neural_network_np(data_tensor: np.ndarray, weight_bias_tensor) -> np.ndarray:
    """
    Simulates the neural network by performing forward pass and applying sigmoid activation function.

    Args:
        data_tensor (np.ndarray): The input tensor to be passed through the neural network.
        weight_bias_tensor (tuple[np.ndarray, float]): A tuple containing the weights tensor and bias value of the neural network.

    Returns:
        np.ndarray: A 2D array containing the output of the sigmoid activation function for each input data point.

    Examples:
        >>> import numpy as np
        >>> data_tensor = np.random.rand(10, 5)
        >>> weight_bias_tensor = (np.random.rand(5), 1.0)
        >>> sigmoid_tensor = simulate_neural_network_np(data_tensor, weight_bias_tensor)

    Notes:
        The neural network is assumed to have one layer with the given weights and bias.
        The output of the sigmoid activation function is rounded to three decimal places.
    """
    # Perform forward pass by multiplying input data tensor with weights
    output_tensor: np.ndarray = data_tensor * weight_bias_tensor[0].reshape(1, -1)

    # Extract the bias value from the input tuple
    bias: float = weight_bias_tensor[1][0]

    # Calculate the weighted sum of inputs and add bias
    result: np.ndarray = output_tensor.sum(axis=1).reshape(-1, 1)

    # Apply sigmoid activation function to get output values and round them to three decimal places
    sigmoid_tensor: np.ndarray = np.array(list(map(lambda r: np.round(1 / (1 + np.exp(-(r + bias))), 3), result)))
    
    return sigmoid_tensor
    

##### 1. Инициализировали модель и получили из нее веса

In [7]:
model_logic: Sequential = create_model()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ output (Dense)                  │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4 (16.00 B)

 Trainable params: 4 (16.00 B)

 Non-trainable params: 0 (0.00 B)

In [8]:
weights_tensor: np.ndarray = model_logic.get_layer('output').get_weights()
weights_tensor

[array([[1.0692016 ],
        [0.10062623],
        [0.58711576]], dtype=float32),
 array([0.], dtype=float32)]

##### 2. Прогнать датасет через не обученную модель и реализованные 2 функции. Сравнить результат.

In [9]:
data_tensor_np: np.ndarray = X.values

In [10]:
output_tensor, sigmoid = simulate_neural_network(data_tensor_np, weights_tensor)
output_tensor_np = simulate_neural_network_np(data_tensor_np, weights_tensor)

In [11]:
print("Входные данные:")
print(data_tensor_np)
print("\nТензор весов:")
print(weights_tensor)
print("\nВыход после симуляции:")
print(output_tensor)
print("\nВыход sigmoid:")
print(np.array(sigmoid).reshape(-1, 1))
print("\nВыход sigmoid после симуляции NP:")
print(output_tensor_np)

Входные данные:
[[0 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 1]
 [1 0 0]
 [1 0 1]
 [1 1 0]
 [1 1 1]]

Тензор весов:
[array([[1.0692016 ],
       [0.10062623],
       [0.58711576]], dtype=float32), array([0.], dtype=float32)]

Выход после симуляции:
[[0.         0.         0.        ]
 [0.         0.         0.58711576]
 [0.         0.10062623 0.        ]
 [0.         0.10062623 0.58711576]
 [1.06920159 0.         0.        ]
 [1.06920159 0.         0.58711576]
 [1.06920159 0.10062623 0.        ]
 [1.06920159 0.10062623 0.58711576]]

Выход sigmoid:
[[0.5  ]
 [0.643]
 [0.525]
 [0.665]
 [0.744]
 [0.84 ]
 [0.763]
 [0.853]]

Выход sigmoid после симуляции NP:
[[0.5  ]
 [0.643]
 [0.525]
 [0.665]
 [0.744]
 [0.84 ]
 [0.763]
 [0.853]]


In [12]:
predictions = model_logic.predict(X)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


In [13]:
predictions

array([[0.5       ],
       [0.6427031 ],
       [0.52513534],
       [0.6654644 ],
       [0.744445  ],
       [0.839743  ],
       [0.76311386],
       [0.8528265 ]], dtype=float32)

In [14]:
predictions - output_tensor_np

array([[ 0.        ],
       [-0.00029688],
       [ 0.00013534],
       [ 0.0004644 ],
       [ 0.00044503],
       [-0.00025698],
       [ 0.00011386],
       [-0.00017352]])

In [15]:
predictions - np.array(sigmoid).reshape(-1, 1)

array([[ 0.        ],
       [-0.00029688],
       [ 0.00013534],
       [ 0.0004644 ],
       [ 0.00044503],
       [-0.00025698],
       [ 0.00011386],
       [-0.00017352]])

По результатам предсказания функций и модели  совпадают (одинаковые веса), отличаются из-за того что присутствует округление

##### 3. Обучить модель и получить веса после обучения

In [16]:
learn_model: Sequential = create_error_report(model_logic)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 518ms/step - accuracy: 0.5000 - loss: 0.8334 - val_accuracy: 0.0000e+00 - val_loss: 1.6753
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5000 - loss: 0.8328 - val_accuracy: 0.0000e+00 - val_loss: 1.6725
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5000 - loss: 0.8321 - val_accuracy: 0.0000e+00 - val_loss: 1.6696
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5000 - loss: 0.8315 - val_accuracy: 0.0000e+00 - val_loss: 1.6668
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5000 - loss: 0.8309 - val_accuracy: 0.0000e+00 - val_loss: 1.6639
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5000 - loss: 0.8302 - val_accuracy: 0.0000e+00 - val_loss: 1.6611
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5000 - loss: 0.8296 - val_accuracy: 0.0000e+00 - val_loss: 1.6583
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.5000 - loss: 0.8290 - val

In [17]:
learn_weights: np.ndarray = learn_model.get_weights()
learn_weights

[array([[0.9717244 ],
        [0.01413314],
        [0.4895458 ]], dtype=float32),
 array([-0.09750675], dtype=float32)]

##### 4.	Прогнать датасет через обученную модель и реализованные 2 функции. Сравнить результат.

In [18]:
output_tensor, sigmoid = simulate_neural_network(data_tensor_np, learn_weights)
output_tensor_np = simulate_neural_network_np(data_tensor_np, learn_weights)

In [19]:
print("Входные данные:")
print(data_tensor_np)
print("\nТензор весов:")
print(weights_tensor)
print("\nВыход после симуляции:")
print(output_tensor)
print("\nВыход sigmoid:")
print(np.array(sigmoid).reshape(-1, 1))
print("\nВыход sigmoid после симуляции NP:")
print(output_tensor_np)

Входные данные:
[[0 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 1]
 [1 0 0]
 [1 0 1]
 [1 1 0]
 [1 1 1]]

Тензор весов:
[array([[1.0692016 ],
       [0.10062623],
       [0.58711576]], dtype=float32), array([0.], dtype=float32)]

Выход после симуляции:
[[0.         0.         0.        ]
 [0.         0.         0.48954579]
 [0.         0.01413314 0.        ]
 [0.         0.01413314 0.48954579]
 [0.97172439 0.         0.        ]
 [0.97172439 0.         0.48954579]
 [0.97172439 0.01413314 0.        ]
 [0.97172439 0.01413314 0.48954579]]

Выход sigmoid:
[[0.476]
 [0.597]
 [0.479]
 [0.6  ]
 [0.706]
 [0.796]
 [0.709]
 [0.799]]

Выход sigmoid после симуляции NP:
[[0.476]
 [0.597]
 [0.479]
 [0.6  ]
 [0.706]
 [0.796]
 [0.709]
 [0.799]]


In [20]:
predictions = learn_model.predict(X)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


In [21]:
predictions

array([[0.4756426 ],
       [0.59677345],
       [0.47916865],
       [0.60016966],
       [0.70562255],
       [0.7963707 ],
       [0.70854974],
       [0.79865295]], dtype=float32)

In [22]:
predictions - output_tensor_np

array([[-0.00035741],
       [-0.00022655],
       [ 0.00016865],
       [ 0.00016966],
       [-0.00037745],
       [ 0.00037069],
       [-0.00045026],
       [-0.00034705]])

In [23]:
predictions - np.array(sigmoid).reshape(-1, 1)

array([[-0.00035741],
       [-0.00022655],
       [ 0.00016865],
       [ 0.00016966],
       [-0.00037745],
       [ 0.00037069],
       [-0.00045026],
       [-0.00034705]])

По результатам предсказания функций и модели совпадают (одинаковые веса), отличаются из-за того что присутствует округление
